# SpanMarker cross-validation training
This notebook performs 5-fold crossvalidation for a SpanMarker model. Parquet files must be stored in the same directory. An internet connection is required to download the germeval2014 dataset necessary for feature mapping.

The evaluation is performed separately in `cv_model_evaluation.ipynb`.

In [ ]:
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)


import numpy as np
from customized_spanmarker_training import NoTrainPreprocTrainer, preprocess_dataset
from datasets import Dataset, DatasetDict, concatenate_datasets, load_dataset
from sklearn.model_selection import KFold
from span_marker import SpanMarkerModel
from span_marker.label_normalizer import AutoLabelNormalizer
from torch.optim import AdamW
from transformers import TrainingArguments, get_scheduler

Loading datasets remotely and from parquet.

In [5]:
germeval = load_dataset("gwlms/germeval2014")["train"]
germeval = germeval.select_columns(["tokens", "ner_tags"])
krp_19jhd = Dataset.from_parquet("krp_19jhd.parquet")
krp_20jhd = Dataset.from_parquet("krp_20jhd.parquet")
rrb_19jhd = Dataset.from_parquet("rrb_19jhd.parquet")
rrb_20jhd = Dataset.from_parquet("rrb_20jhd.parquet")
gszh = Dataset.from_parquet("gszh.parquet")

Using the latest cached version of the module from /Users/adrianvanderlek/.cache/huggingface/modules/datasets_modules/datasets/gwlms--germeval2014/66c688527595f251c65f33876dfb2dee0ae258c4f5df3a6e908f3511559bf4c3 (last modified on Fri Jan  5 13:59:07 2024) since it couldn't be found locally at gwlms/germeval2014, or remotely on the Hugging Face Hub.


Mapping features to the original germeval2014 indices.

In [ ]:
krp_19jhd = krp_19jhd.map(features=germeval.features)
krp_20jhd = krp_20jhd.map(features=germeval.features)
rrb_19jhd = rrb_19jhd.map(features=germeval.features)
rrb_20jhd = rrb_20jhd.map(features=germeval.features)
gszh = gszh.map(features=germeval.features)

Training.

In [8]:
# Overarching training parameters
gradient_accumulation_steps = 2
train_batch_size = 4
n_epochs = 10

# Cross-validation with custom stratified KFold over all five datasets
kf = KFold(n_splits=5)

for i, (
    (krp_19jhd_train_idx, krp_19jhd_eval_idx),
    (krp_20jhd_train_idx, krp_20jhd_eval_idx),
    (rrb_19jhd_train_idx, rrb_19jhd_eval_idx),
    (rrb_20jhd_train_idx, rrb_20jhd_eval_idx),
    (gszh_train_idx, gszh_eval_idx),
) in enumerate(
    list(
        zip(
            kf.split(np.zeros(krp_19jhd.num_rows)),
            kf.split(np.zeros(krp_20jhd.num_rows)),
            kf.split(np.zeros(rrb_19jhd.num_rows)),
            kf.split(np.zeros(rrb_20jhd.num_rows)),
            kf.split(np.zeros(gszh.num_rows)),
        )
    )
):
    print(f"Fold {i}:")

    # Selecting the training subfolds
    krp_19jhd_train = krp_19jhd.select(krp_19jhd_train_idx)
    krp_20jhd_train = krp_20jhd.select(krp_20jhd_train_idx)
    rrb_19jhd_train = rrb_19jhd.select(rrb_19jhd_train_idx)
    rrb_20jhd_train = rrb_20jhd.select(rrb_20jhd_train_idx)
    gszh_train = gszh.select(gszh_train_idx)

    # Selecting the test subfolds
    krp_19jhd_eval = krp_19jhd.select(krp_19jhd_eval_idx)
    krp_20jhd_eval = krp_20jhd.select(krp_20jhd_eval_idx)
    rrb_19jhd_eval = rrb_19jhd.select(rrb_19jhd_eval_idx)
    rrb_20jhd_eval = rrb_20jhd.select(rrb_20jhd_eval_idx)
    gszh_eval = gszh.select(gszh_eval_idx)

    # Concatenating training subfolds
    train_fold = concatenate_datasets(
        [krp_19jhd_train, krp_20jhd_train, rrb_19jhd_train, rrb_20jhd_train, gszh_train]
    ).shuffle(
        seed=42
    )

    # Concatenating test subfolds
    eval_fold = concatenate_datasets(
        [krp_19jhd_eval, krp_20jhd_eval, rrb_19jhd_eval, rrb_20jhd_eval, gszh_eval]
    )

    # Creating fold dataset
    fold_dataset = DatasetDict(
        {
            "train": train_fold,
            "eval": eval_fold,
            "eval_krp_19jhd": krp_19jhd_eval,
            "eval_krp_20jhd": krp_20jhd_eval,
            "eval_rrb_19jhd": rrb_19jhd_eval,
            "eval_rrb_20jhd": rrb_20jhd_eval,
            "eval_gszh": gszh_eval,
        }
    )

    # Model instantiation
    encoder_id = "stefan-it/span-marker-gelectra-large-germeval14"
    model = SpanMarkerModel.from_pretrained(
        # Required arguments
        encoder_id,
        # Optional arguments
        model_max_length=256,
        entity_max_length=8,
    )

    # Training arguments
    args = TrainingArguments(
        output_dir="models/span-marker-ktzh-stazh-cv/tmp",
        learning_rate=1e-05,
        gradient_accumulation_steps=gradient_accumulation_steps,
        per_device_train_batch_size=train_batch_size,
        per_device_eval_batch_size=4,
        num_train_epochs=n_epochs,
        evaluation_strategy="steps",
        save_strategy="no",
        eval_steps=1500,
        push_to_hub=False,
        logging_steps=50,
        warmup_ratio=0.05,
    )

    # Preprocessing dataset
    train_dataset = preprocess_dataset(
        model,
        fold_dataset["train"],
        AutoLabelNormalizer.from_config(model.config),
        model.tokenizer,
        dataset_name="train",
        is_evaluate=False,
    )

    
    # Setting up learning rate scheduler
    num_training_steps = int(
        len(train_dataset) / gradient_accumulation_steps / train_batch_size * n_epochs
    )

    optimizer = AdamW(model.parameters(), lr=1e-05)

    l_r_scheduler = get_scheduler(
        "polynomial",
        optimizer,
        num_warmup_steps=int(0.05 * num_training_steps),
        num_training_steps=num_training_steps,
        scheduler_specific_kwargs=dict(lr_end=5e-07, power=3),
    )

    # Instantiating trainer
    trainer = NoTrainPreprocTrainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=fold_dataset["eval"],
        optimizers=(optimizer, l_r_scheduler),
    )

    # Training and saving separate model per fold
    trainer.train()
    trainer.save_model(f"models/span-marker-ktzh-stazh-cv/fold_{i}")

    del trainer
    del model

Fold 0:


/Users/adrianvanderlek/anaconda3/envs/ner2/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Step,Training Loss,Validation Loss,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1500,0.003000,0.004730,0.907415,0.941786,0.924281,0.989128
3000,0.002300,0.004246,0.927280,0.940590,0.933888,0.991111
4500,0.001200,0.004632,0.925623,0.947767,0.936564,0.991401
6000,0.001300,0.004906,0.923821,0.952552,0.937966,0.991668
7500,0.000200,0.005432,0.930051,0.948963,0.939412,0.991802
9000,0.000800,0.005429,0.923997,0.954944,0.939216,0.991802
10500,0.000400,0.005449,0.926299,0.952153,0.939048,0.991935
12000,0.000300,0.005776,0.926659,0.952153,0.939233,0.991935


Label normalizing the evaluation dataset:   0%|          | 0/1632 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Tokenizing the evaluation dataset:   0%|          | 0/1627 [00:00<?, ? examples/s]

This SpanMarker model won't be able to predict 5.533597% of all annotated entities in the evaluation dataset. This is caused by the SpanMarkerModel maximum model input length of 256 tokens.
A total of 140 (5.533597%) entities were missed due to the maximum input length.


Spreading data between multiple samples:   0%|          | 0/1627 [00:00<?, ? examples/s]

Using the latest cached version of the module from /Users/adrianvanderlek/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--seqeval/541ae017dc683f85116597d48f621abc7b21b88dc42ec937c71af5415f0af63c (last modified on Fri Jan  5 15:34:57 2024) since it couldn't be found locally at evaluate-metric--seqeval, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from /Users/adrianvanderlek/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--seqeval/541ae017dc683f85116597d48f621abc7b21b88dc42ec937c71af5415f0af63c (last modified on Fri Jan  5 15:34:57 2024) since it couldn't be found locally at evaluate-metric--seqeval, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from /Users/adrianvanderlek/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--seqeval/541ae017dc683f85116597d48f621abc7b21b88dc42ec937c71af5415f0af63c (last modified on Fri Jan  5 15:34:57 2024) since it couldn't 